<h2 align="center"> Deploying Models with TensorFlow Serving and Docker</h2>

###  Loading and Preprocess Data

In [19]:
%%writefile -a train.py
import os
import time
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

Writing train.py


In [2]:
#Souce: https://www.kaggle.com/snap/amazon-fine-food-reviews/data
df=pd.read_csv('train.csv')
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,184502,B001BCVY4W,A1JMR1N9NBYJ1X,Mad Ethyl Flint,0,0,4,1228176000,Doesn't look like catfood!,"When you first open the can, it looks like som..."
1,182779,B0052LZ6XI,A2CVFBDRXYFZG9,vanostran,0,0,5,1335657600,Solid Mayo,This is a solid mayo. Will not disappoint. At ...
2,193862,B005IW4WFY,A26TWY9AD935HC,S. Finefrock,1,1,4,1321488000,"Smart, healthy food",Gone are the days that cold pizza washed down ...
3,198524,B000FVBYCW,A34E1744VPQCNU,Dbp323,1,1,5,1337299200,Dbp323,Recommend it to all. Lovely natural sweet del...
4,264525,B001QW06UA,AV9ZR8HC36LD3,BGL,5,5,5,1298764800,Innova Cat Food,This is the only cat food my two nine year old...


In [20]:
%%writefile -a train.py

def load_dataset(file_path, num_samples):
    df = pd.read_csv(file_path, usecols=[6, 9], nrows=num_samples)
    df.columns = ['rating', 'title']

    text = df['title'].tolist()
    text = [str(t).encode('ascii', 'replace') for t in text]
    text = np.array(text, dtype=object)[:]
    
    labels = df['rating'].tolist()
    labels = [1 if i>=4 else 0 if i==3 else -1 for i in labels]
    labels = np.array(pd.get_dummies(labels), dtype=int)[:] 

    return labels, text


Appending to train.py


In [4]:
tmp_labels, tmp_text =load_dataset('train.csv', 100)
tmp_text.shape

(100,)

### Building the Classification Model using TF Hub

In [21]:
%%writefile -a train.py
##https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1
##https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1

def get_model():
    hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", output_shape=[128], 
                           input_shape=[], dtype=tf.string, name='input', trainable=False)

    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(3, activation='softmax', name='output'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='Adam', metrics=['accuracy'])
    model.summary()
    return model

Appending to train.py


In [6]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")
embeddings = embed(["this is a test", "look at the embeddings"])
embeddings

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.05650096,  0.2567145 ,  0.24404189,  0.14395264, -0.05569138,
        -0.10513686,  0.09544804,  0.3080969 , -0.218672  , -0.03048538,
        -0.19036277,  0.01005417,  0.11541115, -0.14860378,  0.03914931,
        -0.2561884 , -0.15442336,  0.12836292,  0.0469152 , -0.1500514 ,
        -0.13068351, -0.01958708,  0.09192695,  0.1208052 , -0.12291992,
        -0.04548305, -0.3679261 ,  0.05125156,  0.09797382, -0.10217863,
        -0.1965521 ,  0.15523128, -0.05881735, -0.16426983,  0.06646369,
         0.05789638,  0.15421619, -0.24014738,  0.11075415, -0.10756174,
        -0.01679449, -0.01877424,  0.18602087,  0.2623015 , -0.3829217 ,
        -0.34895867, -0.0868978 ,  0.02295742,  0.03787762, -0.02646483],
       [-0.01533648,  0.2517981 ,  0.15771465,  0.10011643, -0.03027005,
        -0.09655963,  0.10035348, -0.13405894, -0.13515756,  0.15999079,
        -0.0257801 ,  0.01482286,  0.17336626,  0.02416893, -0.02589497,
 

### Defining Training Procedure

In [22]:
%%writefile -a train.py

def train(EPOCHS=6, BATCH_SIZE=32, TRAIN_FILE='train.csv', VAL_FILE='test.csv'):
    WORKING_DIR = os.getcwd() #use to specify model checkpoint path
    print("Loading training/validation data ...")
    y_train, x_train = load_dataset(TRAIN_FILE, num_samples=100000)
    y_val, x_val = load_dataset(VAL_FILE, num_samples=10000)

    print("Training the model ...")
    model = get_model()
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1,
              validation_data=(x_val, y_val),
              callbacks=[tf.keras.callbacks.ModelCheckpoint(os.path.join(WORKING_DIR,
                                                                         'model_checkpoint'),
                                                            monitor='val_loss', verbose=1,
                                                            save_best_only=True,
                                                            save_weights_only=False,
                                                            mode='auto')])
    return model

Appending to train.py


### Training and Exporting Model as Protobuf

In [8]:
%%writefile -a train.py

def export_model(model, base_path="amazon_review/"):
    path = os.path.join(base_path, str(int(time.time())))
    tf.saved_model.save(model, path)


if __name__== '__main__':
    model = train()
    export_model(model)

Loading training/validation data ...
Training the model ...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (KerasLayer)           (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
output (Dense)               (None, 3)                 195       
Total params: 124,651,139
Trainable params: 8,451
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/6
3125/3125 [==============================] - ETA: 0s - loss: 0.5315 - accuracy: 0.8018
Epoch 00001: val_loss improved from inf to 0.50801, saving model to C:\Users\Harneet singh\Desktop\Project\new\model_checkpoint
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


...

INFO:tensorflow:Assets written to: C:\Users\Harneet singh\Desktop\Project\new\model_checkpoint\assets


3125/3125 [==============================] - 1054s 337ms/step - loss: 0.4950 - accuracy: 0.8120 - val_loss: 0.4965 - val_accuracy: 0.8123
Epoch 4/6
3125/3125 [==============================] - ETA: 0s - loss: 0.4882 - accuracy: 0.8151
Epoch 00004: val_loss did not improve from 0.49651
3125/3125 [==============================] - 1242s 397ms/step - loss: 0.4882 - accuracy: 0.8151 - val_loss: 0.4972 - val_accuracy: 0.8089
Epoch 5/6
3125/3125 [==============================] - ETA: 0s - loss: 0.4837 - accuracy: 0.8173
Epoch 00005: val_loss did not improve from 0.49651
3125/3125 [==============================] - 1053s 337ms/step - loss: 0.4837 - accuracy: 0.8173 - val_loss: 0.5121 - val_accuracy: 0.8056
Epoch 6/6
3125/3125 [==============================] - ETA: 0s - loss: 0.4778 - accuracy: 0.8189
Epoch 00006: val_loss improved from 0.49651 to 0.49023, saving model to C:\Users\Harneet singh\Desktop\Project\new\model_checkpoint
INFO:tensorflow:Assets written to: C:\Users\Harneet singh\Des

INFO:tensorflow:Assets written to: C:\Users\Harneet singh\Desktop\Project\new\model_checkpoint\assets


3125/3125 [==============================] - 1091s 349ms/step - loss: 0.4778 - accuracy: 0.8189 - val_loss: 0.4902 - val_accuracy: 0.8147
INFO:tensorflow:Assets written to: amazon_review/1600446899\assets


INFO:tensorflow:Assets written to: amazon_review/1600446899\assets


### Test Model

#### Negative Review:

In [14]:
test_sentence = "horrible book, waste of time"
model.predict([test_sentence])

array([[0.87231785, 0.00459841, 0.12308376]], dtype=float32)

#### Positive Review:

In [15]:
test_sentence = "Awesome product."
model.predict([test_sentence])

array([[0.07552193, 0.01465404, 0.9098241 ]], dtype=float32)

### TensorFlow Serving with Docker

`docker pull tensorflow/serving`

`docker run -p 8500:8500 \
            -p 8501:8501 \
            --mount type=bind,\
            source=amazon_review/,\
            target=/models/amazon_review \
            -e MODEL_NAME=amazon_review \
            -t tensorflow/serving`

###  Setting up a REST Client to Perform Model Predictions

#### Perform Model Prediction

##### Support for gRPC and REST

- TensorFlow Serving supports
    - Remote Procedure Protocal (gRPC)
    - Representational State Transfer (REST)
- Consistent API structures
- Server supports both standards simultaneously
- Default ports:
    - RPC: 8500
    - REST: 8501

#### Predictions via REST

- Standard HTTP POST requests
- Response is a JSON body with the prediction
- Request from the default or specific model

Default URI scheme:

`http://{HOST}:{PORT}/v1/models/{MODEL_NAME}`

Specific model versions:

`http://{HOST}:{PORT}/v1/models/{MODEL_NAME}[/versions/{MODEL_VERSION}]:predict`

In [16]:
%%writefile tf_serving_rest_client.py
import json
import requests
import sys

def get_rest_url(model_name, host='127.0.0.1', port='8501', verb='predict', version=None):
    """ generate the URL path"""
    url = "http://{host}:{port}/v1/models/{model_name}".format(host=host, port=port, model_name=model_name)
    if version:
        url += 'versions/{version}'.format(version=version)
    url += ':{verb}'.format(verb=verb)
    return url


def get_model_prediction(model_input, model_name='amazon_review', signature_name='serving_default'):
    """ no error handling at all, just poc"""

    url = get_rest_url(model_name)
    #In the row format, inputs are keyed to instances key in the JSON request.
    #When there is only one named input, specify the value of instances key to be the value of the input:
    data = {"instances": [model_input]}
    
    rv = requests.post(url, data=json.dumps(data))
    if rv.status_code != requests.codes.ok:
        rv.raise_for_status()
    
    return rv.json()['predictions']

if __name__ == '__main__':

    print("\nGenerate REST url ...")
    url = get_rest_url(model_name='amazon_review')
    print(url)
    
    while True:
        print("\nEnter an Amazon review [:q for Quit]")
        if sys.version_info[0] <= 3:
            sentence = input()
        if sentence == ':q':
            break
        model_input = sentence
        model_prediction = get_model_prediction(model_input)
        print("The model predicted ...")
        print(model_prediction)

Overwriting tf_serving_rest_client.py


###  Setting up a gRPC Client to Perform Model Predictions

Modified from [https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_client.py](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_client.py#L152)

#### Predictions via gRPC

More sophisticated client-server connections

- Prediction data has to be converted to the Protobuf format
- Request types have designated types, e.g. float, int, bytes
- Payloads need to be converted to base64
- Connect to the server via gRPC stubs

#### gRPC vs REST: When to use which API standard

- Rest is easy to implement and debug
- RPC is more network efficient, smaller payloads
- RPC can provide much faster inferences!

In [17]:
%%writefile tf_serving_grpc_client.py
import sys
import grpc
from grpc.beta import implementations
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2, get_model_metadata_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc


def get_stub(host='127.0.0.1', port='8500'):
    channel = grpc.insecure_channel('127.0.0.1:8500') 
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    return stub


def get_model_prediction(model_input, stub, model_name='amazon_review', signature_name='serving_default'):
    """ no error handling at all, just poc"""
    request = predict_pb2.PredictRequest()
    request.model_spec.name = model_name
    request.model_spec.signature_name = signature_name
    request.inputs['input_input'].CopyFrom(tf.make_tensor_proto(model_input))
    response = stub.Predict.future(request, 5.0)  # 5 seconds
    return response.result().outputs["output"].float_val


def get_model_version(model_name, stub):
    request = get_model_metadata_pb2.GetModelMetadataRequest()
    request.model_spec.name = 'amazon_review'
    request.metadata_field.append("signature_def")
    response = stub.GetModelMetadata(request, 10)
    # signature of loaded model is available here: response.metadata['signature_def']
    return response.model_spec.version.value

if __name__ == '__main__':
    print("\nCreate RPC connection ...")
    stub = get_stub()
    while True:
        print("\nEnter an Amazon review [:q for Quit]")
        if sys.version_info[0] <= 3:
            sentence = raw_input() if sys.version_info[0] < 3 else input()
        if sentence == ':q':
            break
        model_input = [sentence]
        model_prediction = get_model_prediction(model_input, stub)
        print("The model predicted ...")
        print(model_prediction)

Overwriting tf_serving_grpc_client.py


In [18]:
!saved_model_cli show --dir /home/cicada/Downloads/rhyme/TF_Serving/amazon_review/1597906549 --all

2020-09-18 22:07:46.175203: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2020-09-18 22:07:46.175561: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Traceback (most recent call last):
  File "c:\users\harneet singh\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\harneet singh\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Harneet singh\anaconda3\Scripts\saved_model_cli.exe\__main__.py", line 7, in <module>
  File "c:\users\harneet singh\anaconda3\lib\site-packages\tensorflow\python\tools\saved_model_cli.py", line 1185, in main
    args.func(args)
  File "c:\users\harneet singh\anaconda3\lib\site-packages\tensorflow\python\tools\saved_model_cli.py", line 715, in show
    _show_all(args